In [ ]:
import re

from nipype.utils.draw_gantt_chart import log_to_dict
from numpy import around, polyfit

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def create_arrays(list_of_logs):
    log_dicts = [get_mem_usage(log) for log in list_of_logs]
    curves = {}
    for log in log_dicts:
        for key in log:
            if key not in curves:
                curves[key] = {'x': [], 'y': []}
            for log in log_dicts:
                if key in log and log[key]['TRs'] not in curves[key]['x']:
                    curves[key]['x'].append(log[key]['TRs'])
                    curves[key]['y'].append(log[key]['runtime_memory_gb'])
    return curves


def get_mem_usage(log, TRs=None):
    if TRs is None:
        TRs = int(log.rstrip('.log'))
    q = [i for i in log_to_dict(log) if
         'num_threads' in i
        ]
    r = {i: {
        'runtime_memory_gb': max([j.get('runtime_memory_gb', 0) for j in q if i in j['id']]),
        'estimated_memory_gb': max([j.get('estimated_memory_gb', 0) for j in q if i in j['id']]),
        'TRs': TRs
    } for i in {
        (re.search('.*(?=_\d*)', i['id'].split('.')[-1]).group() if
         re.search('.*(?=_\d*)', i['id'].split('.')[-1]) else
         i['id'].split('.')[-1]) for i in q
    }}
    return dict(sorted(r.items(), key=lambda item: item[1]['runtime_memory_gb'], reverse=True))


def calculate_formula(coordinates):
    '''
    Parameters
    ----------
    coordinates : tuple of lists
        ([x1, x2, x3], [y1, y2, y3])
    '''
    a, b = around(polyfit(*coordinates, 1), 6)
    a2, b2, c2 = around(polyfit(*coordinates, 2), 6)
    return f'{a}x + {b}\n\t{a2}x² + {b2}x + {c2}'


def calculate_formulas(arrays):
    for key in arrays:
        print(f'{key}:\n\t{calculate_formula((arrays[key]["x"], arrays[key]["y"]))}')

In [ ]:
calculate_formulas(create_arrays(['300.log', '600.log']))